In [1]:
import heapq
from collections import defaultdict, deque
from typing import Dict, List

**53. Maximum Subarray**


In [ ]:
class Solution:
    # Kadane's algo:
    def maxSubArray(self, nums: List[int]) -> int:  # 44% time, 6% memory
        """
        At index i store max(max subarray up to i plus the value of i, the value of i)
        So only considering one thing at a time becomes a lot easier
        Base case: [] -> 0
        Base case: [n] -> [n] -> n
        Case: [n, m] -> [n, max(n + m, m)] -> max(n, n+m, m) essentially
        """

        dp = [nums[0]] + [
            0 for _ in range(len(nums) - 1)
        ]  # -1 because we init the first one already

        for i in range(1, len(nums)):
            dp[i] = max(dp[i - 1] + nums[i], nums[i])

        return max(dp)

    def maxSubArraySpaceOptimized(self, nums: List[int]) -> int:  # 13% time, 10% memory
        """
        I can see that dp[i] is only reliant on dp[i-1] so we don't actually need an array

        There is only one parameter: i (index through nums)
        Bounds: [0, len(nums)-1]
        Direction: doesn't matter since it is only one parameter without restrictions
        """

        dp = nums[0]
        res = dp
        for i in range(1, len(nums)):
            dp = max(dp, 0) + nums[i]
            res = max(res, dp)

        return res

    def maxSubArrayDivideAndConquer(self, nums: List[int]) -> int:
        """
        Useless af lol, it's just doing Kadane's algo more than once for no reason
        """

        def dfs(l, r):
            if l > r:
                return float("-inf")

            m = (l + r) // 2
            leftSum = rightSum = curSum = 0
            for i in range(m - 1, l - 1, -1):
                curSum += nums[i]
                leftSum = max(leftSum, curSum)

            curSum = 0
            for i in range(m + 1, r + 1):
                curSum += nums[i]
                rightSum = max(rightSum, curSum)

            return max(dfs(l, m - 1), dfs(m + 1, r), leftSum + nums[m] + rightSum)

        return dfs(0, len(nums) - 1)


maxSubArray = Solution()
print(maxSubArray.maxSubArraySpaceOptimized([-2, 1, -3, 4, -1, 2, 1, -5, 4]))
print(maxSubArray.maxSubArraySpaceOptimized([1]))
print(maxSubArray.maxSubArraySpaceOptimized([5, 4, -1, 7, 8]))

6
1
23


**55. Jump Game**

Very greedy algo lol


In [ ]:
class Solution:  # 95% time, 8% memory
    def canJump(self, nums: List[int]) -> bool:
        """
        Iterate in reverse
        whenever you see a zero start a counter, if you reach a number that is > the counter you can reset the counter

        NOTE: the last number of the array is irrelevant
        """
        counter = 0
        for i in reversed(range(len(nums) - 1)):
            if nums[i] > counter:
                counter = 0
            else:
                counter += 1
        return counter == 0


canJump = Solution()
print(canJump.canJump([3, 2, 1, 0, 4]))
print(canJump.canJump([2, 3, 1, 1, 4]))

False
True


**45. Jump Game II**

Each element nums[i] represents the maximum length of a forward jump from index i

Return the minimum number of jumps to reach nums[n - 1]. The test cases are generated such that you can reach nums[n - 1].


In [ ]:
class Solution:
    def jump(self, nums: List[int]) -> int:  # 20% time, 12 % memory
        """
        at index i, update dp[i + x] x in [1,nums[i]] with min(dp[i+x], dp[i] + 1)

        DP Bottom Up solution O(n^2) time O(n) space
        """

        # default high number (array is only ones)
        dp = [len(nums)] * len(nums)

        # init the starting position
        dp[0] = 0

        for i in range(len(nums)):
            # i + 1 because you are jumping forward only, not up then down where you are
            for j in range(i, min(i + 1 + nums[i], len(nums))):
                dp[j] = min(dp[i] + 1, dp[j])

        return dp[-1]

    def jumpBFSGreedy(self, nums: List[int]) -> int:  # 54% time, 12% memory
        """
        Go as far as you can each time
        """
        res = 0
        left, right = 0, 0

        while right < len(nums) - 1:
            furthest = 0
            for idx in range(left, right + 1):
                # while you move your left pointer up to the current furthest point you KNOW you CAN reach
                # check how far you can go beyond that.
                furthest = max(furthest, idx + nums[idx])
            left = right + 1
            right = furthest
            res += 1
        return res


jump = Solution()
jump.jumpBFSGreedy([2, 3, 1, 1, 4]), jump.jumpBFSGreedy([2, 3, 0, 1, 4])

(2, 2)

**678. Valid Parenthesis String**


In [ ]:
class Solution:
    def checkValidString(self, s: str) -> bool:  # 100% time, 10% memory
        """
        NeetCode, store the minimum and maximum amount of open left bracketes you can have at the current index
        """
        minL, maxL = 0, 0
        for c in s:
            if c == "(":
                minL += 1
                maxL += 1
            elif c == "*":
                minL -= 1
                maxL += 1
            else:
                minL -= 1
                maxL -= 1

            if maxL < 0:
                return False
            if minL < 0:
                # replace some ')' with '' mappings for '*'
                minL = 0
        return min(minL, maxL) == 0

        # """
        # Can have 2 lists:
        # '(' []
        # '*' []
        # Always try to fill ')' with the opening, and then asterisk
        # For whatever remains in the 2 stacks, you need to check if the indices of the * are greater than the opening
        # """
        # op = []
        # ast = deque([])

        # for i in range(len(s)):
        #     if s[i] == "(":
        #         op.append(i)
        #     elif s[i] == "*":
        #         ast.append(i)
        #     else:
        #         # s[i] == ")"
        #         if op:
        #             op.pop()
        #         elif ast:
        #             ast.pop()
        #         else:
        #             return False

        # # Need to check the stacks, they won't be empty if the asterisks need to be used as a ')'
        # for op_i in op:
        #     while ast and ast[0] < op_i:
        #         ast.popleft()
        #     if ast:
        #         continue
        #         # else:
        #     return False
        # # if there are asterisks remaining, they will just be empty space so it is fine
        # return True


checkValidString = Solution()

print(checkValidString.checkValidString("()"))
print(checkValidString.checkValidString("(*)"))
print(checkValidString.checkValidString("(*))"))
print(checkValidString.checkValidString("(*)))"))
print(
    checkValidString.checkValidString(
        "(((((*(()((((*((**(((()()*)()()()*((((**)())*)*)))))))(())(()))())((*()()(((()((()*(())*(()**)()(())"
    )
)

True
True
True
False
False


**1899. Merge Triplets to Form Target Triplet**


In [31]:
class Solution:  # 83% time, 27% memory
    def mergeTriplets(self, triplets: List[List[int]], target: List[int]) -> bool:
        """
        Avoid all triples where one of the integers are greater than their corresponding target integer
        """
        target_counter = [0] * 3
        for triplet in triplets:
            if any(a > b for a, b in zip(triplet, target)):
                # docstring: ignore all triplets that violate the <= target
                continue
            for i in range(len(triplet)):
                if triplet[i] == target[i]:
                    target_counter[i] = triplet[i] == target[i]

            if all(target_counter):
                return True

        return False


mergeTriplets = Solution()
print(mergeTriplets.mergeTriplets([[2, 5, 3], [1, 8, 4], [1, 7, 5]], [2, 7, 5]))
print(mergeTriplets.mergeTriplets([[3, 4, 5], [4, 5, 6]], [3, 2, 5]))
print(
    mergeTriplets.mergeTriplets([[2, 5, 3], [2, 3, 4], [1, 2, 5], [5, 2, 3]], [5, 5, 5])
)

True
False
True


**763. Partition Labels**


In [ ]:
class Solution:
    def partitionLabels(self, s: str) -> List[int]:  # 92% time, 10% memory
        """
        Hashset of seen letters
        Whenever a new letter is seen, start a new substring
        If a letter that has been seen prior is seen, concat all the substrings that need to be concatted
        This means that there need to be multiple hashsets with for each substring so that we can know which hashsets to combine
        """

        # NOTE: this is essentially a stack question

        # store the indices of the start of a substring (the end is demarked by the start of the next substring)
        boundaries: List[int] = []
        # store the index of the first occurrence of the string
        seen: Dict[str, int] = {}
        for i, c in enumerate(s):
            if c in seen:
                first = seen[c]
                # while there is a substring that starts after the start of the new substring, get rid of the smaller substring's start
                # this is how i am concatenating
                while boundaries[-1] > first:
                    boundaries.pop()
            else:
                seen[c] = i
                boundaries.append(i)

        # question asks for the number of characters in each boundary so we need to get the size of the partitions
        # by getting the difference of the start boundaries.
        boundaries.append(len(s))
        for i in range(len(boundaries) - 1):
            boundaries[i] = boundaries[i + 1] - boundaries[i]
        return boundaries[:-1]

    def partitionLabelsNeetCode(self, s: str) -> List[int]:
        """
        Build map of the largest index for each letter
        Iterate through string, updating the largest index of the current substring based on the letters you have seen
            If the index reaches the largest index, you have found the end of a partition so update the result.

        My solution is faster than neetcodes because I only do 1 pass through
        (worst case they are the same since going through boundaries can be O(n) in total)
        but i am on average less than 2 passes, he is always 2 passes.
        """
        lastIndex = {}
        for i, c in enumerate(s):
            lastIndex[c] = i

        res = []
        size = end = 0
        for i, c in enumerate(s):
            size += 1
            end = max(end, lastIndex[c])

            if i == end:
                res.append(size)
                size = 0
        return res


partitionLabels = Solution()
print(partitionLabels.partitionLabels("eccbbbbdc"))
print(partitionLabels.partitionLabels("ababcbacadefegdehijhklij"))
print(partitionLabels.partitionLabels("caedbdedda"))
print(partitionLabels.partitionLabels("qiejxqfnqceocmy"))

[1, 8]
[9, 7, 8]
[1, 9]
[13, 1, 1]


**134. Gas Station**


In [16]:
class Solution:  # 99.5% time, 15% memory
    def canCompleteCircuit(self, gas: List[int], cost: List[int]) -> int:
        """
        They say the solution is unique
        So once you know that there is a solution: sum(gas) >= sum(cost)
        You only need to find the index for which the net path moving forward until the end of the array is > 0
        If you find a starting point that remains positive for a few stops, you ignore all points on that path
        Because none of them would work either since you gave them the best positive start by using the earliest positive gas station
        """
        if sum(gas) < sum(cost):
            return -1

        curPos, curGas = 0, gas[0] - cost[0]  # setup at first index
        # You don't need to setup at first index though.
        # You can set everything to 0 and the if condition will trigger on first iter.
        for i in range(1, len(gas)):
            if curGas < 0:
                # NOTE: modifying curGas <= 0 to curGas < 0 improved time from 77% to 99.5%
                curPos = i  # set starting point to current index, since all earlier indices failed
                curGas = 0
            curGas += gas[i] - cost[i]

        return curPos

    def canCompleteCircuitTwoPointers(
        self, gas: List[int], cost: List[int]
    ) -> int:  # 61% time, 23% memory
        """
        Way more intuitive 2 pointer solution
        It is just trying to clamp the starting and end indices by testing each starting position going backwards
        The direction matters because the question states that it is clockwise
        """
        n = len(gas)
        start, end = n - 1, 0
        tank = gas[start] - cost[start]
        while start > end:
            if tank < 0:
                start -= 1
                tank += gas[start] - cost[start]
            else:
                tank += gas[end] - cost[end]
                end += 1
        return start if tank >= 0 else -1


canCompleteCircuit = Solution()
print(canCompleteCircuit.canCompleteCircuit([1, 2, 3, 4, 5], [3, 4, 5, 1, 2]))
print(canCompleteCircuit.canCompleteCircuit([2, 3, 4], [3, 4, 3]))
print(canCompleteCircuit.canCompleteCircuit([5, 8, 2, 8], [6, 5, 6, 6]))

3
-1
3


**846. Hand of Straights**


In [84]:
class Solution:  # 59% time, 24% memory
    def isNStraightHand(self, hand: List[int], groupSize: int) -> bool:
        """
        NeetCode uses a hashmap counter and a minHeap to keep track of the numbers available

        Whenever the count of a number reaches zero you pop from the minheap because it is necessarily the current minimum in the heap
        Since we start at the minimum value of hand this logic is maintained throughout
        """
        if len(hand) % groupSize:
            return False
        counter = defaultdict(int)
        for n in hand:
            counter[n] += 1
        hand = list(counter.keys())
        heapq.heapify(hand)

        while hand:
            if counter[hand[0]] == 0:
                heapq.heappop(hand)
                continue
            cur = hand[0]

            for i in range(groupSize):
                if not counter.get(cur + i, 0):
                    return False
                else:
                    counter[cur + i] -= 1
        return True


isNStraightHand = Solution()
print(isNStraightHand.isNStraightHand([1, 2, 3, 6, 2, 3, 4, 7, 8], 3))
isNStraightHand.isNStraightHand([3, 4, 7, 4, 6, 3, 6, 5, 2, 8], 2)

True


False

**122. Best Time to Buy and Sell Stock II**


In [ ]:
class Solution:
    def maxProfit(self, prices: List[int]) -> int:  # 100% time, 7% memory
        """
        LeetCode says this is a greedy DP problem

        Can only hold 1 share at a time, so I can make multiple sells if I want

        If you sell on a monotonically increasing stack, your profits should always be maximized
        array: [1,4,7,2,3,1]
        stack: [] -> [1] -> [1,4] -> update result to 3 and popleft -> [4]
        stack: [4, 7] -> update result to 6 and popleft -> [7]
        stack: [7, 2] -> popleft (maintain increasing monotonic queue) -> [2]
        stack: [2, 3] -> res = 7 and popleft -> [3]
        etc

        Turns out we are popping left from deque and appending the current value on every iteration lol
        """

        dq = deque([prices[0]])
        res = 0
        for price in prices[1:]:
            cur = dq.popleft()
            if cur < price:
                res += price - cur
            dq.append(price)
        return res


maxProfit = Solution()
print(maxProfit.maxProfit([7, 1, 5, 3, 6, 4]))
print(maxProfit.maxProfit([1, 2, 3, 4, 5]))
print(maxProfit.maxProfit([7, 6, 4, 3, 1]))

7
4
0


**3191. Minimum Operations to Make Binary Array Elements Equal to One I**


In [ ]:
class Solution:
    # 63% time, 54% memory
    def minOperations(self, nums: List[int]) -> int:
        count = 0
        for left in range(len(nums) - 2):
            if not nums[left]:
                count += 1
                nums[left] ^= 1
                nums[left + 1] ^= 1
                nums[left + 2] ^= 1

        if nums[-2] and nums[-1]:
            return count
        return -1

**2551. Put Marbles in Bags**


In [ ]:
class Solution:  # 56% time, 42% memory
    def putMarbles(self, weights: List[int], k: int) -> int:
        """
        The editorial makes the solution super simple
        It's a Hard because of the observations you need to make to come up with the proper solution
        not because the code is hard.

        The score of a bag (partition) is just the first and last values in it added together.
        So, any value that is first or last has an adjacent pair
            (the last value of the partition before and the first value of the partition after)
        This means you should think of the question in pairs of adjacent values, not in partitions

        Also, the first and last values of the array will necessarily contribute to the score of all combinations of bags.
        There are only k-1 splits for k partitions
        """
        if k == 1:
            return 0
        pairScores = sorted(
            [weights[i] + weights[i + 1] for i in range(len(weights) - 1)]
        )
        return sum(pairScores[-k + 1 :]) - sum(pairScores[: k - 1])